In [ ]:
# the following fixes are highly recommended

#allow oversubscription for openMPI
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"]="true"

# use lower cpu resources for idle codes
from amuse.support import options
options.GlobalOptions.instance().override_value_for_option("polling_interval_in_milliseconds", 10)


This exercise was originally developed together with Selma de Mink for the UVA computational astrophysics course.

Testing the Evolution of Starclusters with Amuse and making predictions for Gaia 
================================================================================

In this exercise you will be guided to use AMUSE to conduct a numerical n-body experiment and make predictions for Gaia. 

The bigger question you will address concerns the question of how stars form. We know that stars typically (and possibly always) form in clusters. However, the birth conditions are very hard to measure.  For example: How "clumpy" or "smooth" are these clusters? How "virialized" are they?  

You will simulate the evolution of star clusters, with different initial conditions. The aim is to investigate if the initial conditions leave imprints that Gaia can observe.  Specifically you will investigate the population of "escapers" and possible "runaway stars", i.e. stars that escape their birth cluster.  To do this we will very closely follow the work by Allison (2012, MNRAS 421, 3338). 

You will report the results of your numerical experiments in the style of a "letter" (= short paper normally on a very timely topic.) Detailed instructions below. The questions in this tutorial are to guide you through the experiment. You can use them for the narrative of your report, but the report should be in the style of a self containing paper, not a numbered list to the questions here.  

(1) Introduction
----------------
a) Read the paper by Allison (2012) and discuss with your peers. What did the author do?  What was the motivation for it? Which terms or concept are new to you?  Look them up.

b) What is the Gaia mission? What will/ has Gaia measured?  What Gaia measurements are relevant for this paper?


(2) Initial conditions
-----------------------

a) Lookup the references given in the paper of the initial conditions for the spatial distribution of stars and their mass distribution. Keep them for reference.

Luckily the initial conditions are already included with AMUSE!

b) Import the fractal model cluster, and read the help to get information on the arguments of this function:


In [ ]:
from amuse.ic.fractalcluster import new_fractal_cluster_model
help(new_fractal_cluster_model)

c) Generate a N=100 model:

In [ ]:
parts=new_fractal_cluster_model(100, do_scale=True)
print(parts)

d) Do you remember from the tutorial what the units are? Confirm that the total mass and potential and kinetic energies match the expected values. (The potential energy calculation is given below - what happens if you do not provide G? can you understand the generated error?)

e) explore the effect of the fractal dimension and virial ratio parameters on the model by plotting the positions and 
velocities of various models. Do you understand the physical meaning of these?

In [ ]:
from amuse.units import units,constants, nbody_system

parts.potential_energy(G=nbody_system.G)

To finish the initial conditions we need to convert the model to physical units, and give the stars a realistic distribution of masses.

f) The Kroupa mass distribution used in the paper can be generated by a a piece-wise power law distribution. Generate a set of masses described by this, and plot the resulting mass distribution.Adapt the parameters to match the paper!

In [ ]:
from amuse.ic.brokenimf import new_broken_power_law_mass_distribution
masses=new_broken_power_law_mass_distribution(1000, mass_boundaries=[0.1,100] | units.MSun, alphas=[-2.35])

g) The way to convert a scaleless model to SI units is by providing a converter. An N-body converter takes two values, fixing the scale of the model (together with the requirement of G=1). (can you understand the units of the converted values below?)

In [ ]:
radius=1 | units.parsec

convert=nbody_system.nbody_to_si( masses.sum(), radius)

print(convert.to_si( 1 | nbody_system.length ))
print(convert.to_nbody( 1 | units.km/units.s ))
print(convert.to_nbody( constants.G ))

h) at this point you should be able to generate intitial conditions matching those of the paper! Do so.

(3) Evolving the models
-----------------------

a) Like the paper mentions, the experiments where done using purely gravitational dynamics. A number of codes for this 
are available in AMUSE. Choose an appropiate one from the tabel given in the AMUSE code paper. 

In [ ]:
from amuse.community.huayno.interface import Huayno

h=Huayno(convert_nbody=convert)
print(h.parameters)
help(h.parameters)

b) write an evolve loop to evolve the model to some specified end_time with some dt, measuring and reporting the energy error 

c) conduct the simulations as reported in the paper. It may be that a simulation takes too long or the energy error is not acceptable ((E-E0)/E0 should be smaller than 0.001). You can try with smaller number of particles, introducing smoothing (epsilon_squared parameter) or change the timestep parameter (usually timestep_parameter), or vary the random seed of the initial distribution (can you figure out the rationale behind these suggestions?)

(4) Analysis
------------

a) reproduce the spatial distribution plot of Allison 2012. Note similarities and differences

b) analyse the kinematics of the escaping stars, discussing Allision 2012.

Explore a bit of parameter space if you feel like it!

(5) Wrapping it up
------------------

a) This exercise is completed by writing short report, see instructions below.  You can distill from the above notebook (maybe copying from the above to a new notebook). Include 1 or 2 figures for the results (matching those from the paper) and maybe 1 or 2 more showing the initial conditions.

b) If you didn't succeed, try to describe why not and what the problems where you ran into. 

c) the orginal paper came out well before the GAIA data releases. Try to find out if anyone has followed up the work with the currently available GAIA data.

d) Bonus points if you extend the model with any of the additional physics mentioned in the paper (stellar evolution, binaries natal gas, but in particular the galactic potential)! But first make sure you full fill the basics. 

Format of your report
---------------------

Write your report as a "letter" (= a brief paper of 4-5 pages max in 2 column format when using the  ``emulateapj`` package. This is a latex class that will give it the appearance of a real ApJ letter. You can download it online. I suggest you use the following latex packages
```
    \\documentclass[apjl]{emulateapj}  % paper will get the appearance of a real ApJ Letter
    \\usepackage{graphicx} % for including plots
    \\usepackage{natbib} % for a proper bibiography
    \\bibliographystyle{apj} % to set the style of the bibliography
    \\usepackage{hyperref} % will make all references in your paper act as hyperlinks in the pdf
```
Make sure you have the following sections and do not exceed 5 pages.
    1. Introduction
    2. Method
    3. Results
    4. Conclusion & Discussion.

Some hints for writing
----------------------

##### Introduction:
A good introduction briefly explains

* the astrophysical context / the bigger question your work adresses / why your work is timely (hint: maybe because it relevant for the data that Gaia will soon release)
* the state of the art and gives credit to most relevant earlier papers (Hint: it would be nice to state that you follow the approach taken by Allison 2012)
* formulate the specific scientific question you are adressing in this work.
* is concise: aim for 0.5-0.75 page in the 2 col format

##### Method

Here you explain which code(s) you are using and you give a very brief summary of what they do providing references. Then state the assumptions and initial conditions you adopt. Typical length is ~1-1.5 page in a letter.

##### Results

Here you describe your main results by systematically dicussing your findings. Hint: It often helps to first decide what the key plots are that you want to show.  Often the result section is often a systematic explanationof what the plots illustate.  Typical length is 2 pages in a letter.

##### Discussion & conclusion 
Summarize your key message. Here you can disuss the implications of your findings, the limitations of your method and possibly state future improvements. Typical length is 0.5 pages in a letter.
